In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 15) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [14, 15, 17]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.440162181854248, Final Batch Loss: 2.2082464694976807
Epoch 2, Loss: 4.431634902954102, Final Batch Loss: 2.1955196857452393
Epoch 3, Loss: 4.43436861038208, Final Batch Loss: 2.2138521671295166
Epoch 4, Loss: 4.429298400878906, Final Batch Loss: 2.202758312225342
Epoch 5, Loss: 4.432542324066162, Final Batch Loss: 2.2178549766540527
Epoch 6, Loss: 4.4314258098602295, Final Batch Loss: 2.2220101356506348
Epoch 7, Loss: 4.422696590423584, Final Batch Loss: 2.209352493286133
Epoch 8, Loss: 4.416726350784302, Final Batch Loss: 2.2054061889648438
Epoch 9, Loss: 4.411235570907593, Final Batch Loss: 2.2046492099761963
Epoch 10, Loss: 4.4143922328948975, Final Batch Loss: 2.2101757526397705
Epoch 11, Loss: 4.403315305709839, Final Batch Loss: 2.195671796798706
Epoch 12, Loss: 4.398383378982544, Final Batch Loss: 2.190040349960327
Epoch 13, Loss: 4.407072067260742, Final Batch Loss: 2.2158682346343994
Epoch 14, Loss: 4.391805171966553, Final Batch Loss: 2.197657823562622
Epoch

Epoch 121, Loss: 2.027718961238861, Final Batch Loss: 0.9879737496376038
Epoch 122, Loss: 2.083704948425293, Final Batch Loss: 1.0287251472473145
Epoch 123, Loss: 1.9738521575927734, Final Batch Loss: 0.9669747352600098
Epoch 124, Loss: 2.032165765762329, Final Batch Loss: 1.0507198572158813
Epoch 125, Loss: 2.105739712715149, Final Batch Loss: 1.0992822647094727
Epoch 126, Loss: 1.9250598549842834, Final Batch Loss: 0.9483267068862915
Epoch 127, Loss: 1.954932153224945, Final Batch Loss: 0.9554834365844727
Epoch 128, Loss: 1.9411250352859497, Final Batch Loss: 0.9258109331130981
Epoch 129, Loss: 2.0324826836586, Final Batch Loss: 1.0531355142593384
Epoch 130, Loss: 1.914914608001709, Final Batch Loss: 0.9776797294616699
Epoch 131, Loss: 1.9749276041984558, Final Batch Loss: 0.9987398982048035
Epoch 132, Loss: 1.9969715476036072, Final Batch Loss: 0.9913132786750793
Epoch 133, Loss: 2.0436244010925293, Final Batch Loss: 1.0102170705795288
Epoch 134, Loss: 1.9552233219146729, Final Batc

Epoch 241, Loss: 1.3535632491111755, Final Batch Loss: 0.7030386924743652
Epoch 242, Loss: 1.3702980279922485, Final Batch Loss: 0.6922874450683594
Epoch 243, Loss: 1.3329049348831177, Final Batch Loss: 0.6790294647216797
Epoch 244, Loss: 1.289788842201233, Final Batch Loss: 0.6131523251533508
Epoch 245, Loss: 1.3759021759033203, Final Batch Loss: 0.6969937086105347
Epoch 246, Loss: 1.358367145061493, Final Batch Loss: 0.6670178174972534
Epoch 247, Loss: 1.385493278503418, Final Batch Loss: 0.770465075969696
Epoch 248, Loss: 1.3358246684074402, Final Batch Loss: 0.6873610019683838
Epoch 249, Loss: 1.2903789281845093, Final Batch Loss: 0.5875228643417358
Epoch 250, Loss: 1.256798267364502, Final Batch Loss: 0.619580090045929
Epoch 251, Loss: 1.3335672616958618, Final Batch Loss: 0.6402621269226074
Epoch 252, Loss: 1.277637243270874, Final Batch Loss: 0.6371472477912903
Epoch 253, Loss: 1.282669186592102, Final Batch Loss: 0.6854606866836548
Epoch 254, Loss: 1.2250239253044128, Final Bat

Epoch 357, Loss: 0.9990724325180054, Final Batch Loss: 0.5215614438056946
Epoch 358, Loss: 1.0800487995147705, Final Batch Loss: 0.5731357932090759
Epoch 359, Loss: 0.9824163317680359, Final Batch Loss: 0.5382736921310425
Epoch 360, Loss: 0.9794229865074158, Final Batch Loss: 0.4740772247314453
Epoch 361, Loss: 0.9346596896648407, Final Batch Loss: 0.4541826546192169
Epoch 362, Loss: 0.9205688834190369, Final Batch Loss: 0.43905705213546753
Epoch 363, Loss: 0.9127344191074371, Final Batch Loss: 0.3824264109134674
Epoch 364, Loss: 1.0033100843429565, Final Batch Loss: 0.4915015697479248
Epoch 365, Loss: 1.0985193252563477, Final Batch Loss: 0.564925491809845
Epoch 366, Loss: 1.0098675191402435, Final Batch Loss: 0.6032668352127075
Epoch 367, Loss: 1.0161621272563934, Final Batch Loss: 0.5790647268295288
Epoch 368, Loss: 1.020173966884613, Final Batch Loss: 0.5213600993156433
Epoch 369, Loss: 0.9244889318943024, Final Batch Loss: 0.4272715151309967
Epoch 370, Loss: 0.9644765555858612, Fi

Epoch 489, Loss: 0.8452869057655334, Final Batch Loss: 0.4414781332015991
Epoch 490, Loss: 0.8302426338195801, Final Batch Loss: 0.4073777496814728
Epoch 491, Loss: 0.8284578621387482, Final Batch Loss: 0.41619402170181274
Epoch 492, Loss: 0.8495442569255829, Final Batch Loss: 0.45137977600097656
Epoch 493, Loss: 0.8466687500476837, Final Batch Loss: 0.42650124430656433
Epoch 494, Loss: 0.8626956045627594, Final Batch Loss: 0.40636932849884033
Epoch 495, Loss: 0.8121715486049652, Final Batch Loss: 0.3614071309566498
Epoch 496, Loss: 0.8299790024757385, Final Batch Loss: 0.3609566390514374
Epoch 497, Loss: 0.8750272691249847, Final Batch Loss: 0.4366535544395447
Epoch 498, Loss: 0.9393079876899719, Final Batch Loss: 0.47879213094711304
Epoch 499, Loss: 0.891334593296051, Final Batch Loss: 0.4050208330154419
Epoch 500, Loss: 0.8612225651741028, Final Batch Loss: 0.46480703353881836
Epoch 501, Loss: 0.8440901041030884, Final Batch Loss: 0.4137289822101593
Epoch 502, Loss: 0.85301029682159

...

Epoch 4508, Loss: 0.2827308624982834, Final Batch Loss: 0.11968103051185608
Epoch 4509, Loss: 0.32745160162448883, Final Batch Loss: 0.1683339923620224
Epoch 4510, Loss: 0.2851405292749405, Final Batch Loss: 0.15529565513134003
Epoch 4511, Loss: 0.2862449884414673, Final Batch Loss: 0.15253616869449615
Epoch 4512, Loss: 0.3190660774707794, Final Batch Loss: 0.15327848494052887
Epoch 4513, Loss: 0.253009170293808, Final Batch Loss: 0.10902726650238037
Epoch 4514, Loss: 0.25849296152591705, Final Batch Loss: 0.09745422005653381
Epoch 4515, Loss: 0.2778814509510994, Final Batch Loss: 0.11486542969942093
Epoch 4516, Loss: 0.2830222398042679, Final Batch Loss: 0.12991821765899658
Epoch 4517, Loss: 0.31190650165081024, Final Batch Loss: 0.14671632647514343
Epoch 4518, Loss: 0.31375719606876373, Final Batch Loss: 0.161209836602211
Epoch 4519, Loss: 0.2589115649461746, Final Batch Loss: 0.12682530283927917
Epoch 4520, Loss: 0.3650791794061661, Final Batch Loss: 0.23541750013828278
Epoch 4521, 

Epoch 4628, Loss: 0.2794994115829468, Final Batch Loss: 0.13618822395801544
Epoch 4629, Loss: 0.2572117894887924, Final Batch Loss: 0.12174546718597412
Epoch 4630, Loss: 0.27354899048805237, Final Batch Loss: 0.13367369771003723
Epoch 4631, Loss: 0.2942562773823738, Final Batch Loss: 0.09936103969812393
Epoch 4632, Loss: 0.27601318061351776, Final Batch Loss: 0.14363504946231842
Epoch 4633, Loss: 0.269070141017437, Final Batch Loss: 0.14906112849712372
Epoch 4634, Loss: 0.26276180893182755, Final Batch Loss: 0.14243581891059875
Epoch 4635, Loss: 0.3446711525321007, Final Batch Loss: 0.22491636872291565
Epoch 4636, Loss: 0.260465145111084, Final Batch Loss: 0.11023594439029694
Epoch 4637, Loss: 0.25081755220890045, Final Batch Loss: 0.1330433338880539
Epoch 4638, Loss: 0.29255741834640503, Final Batch Loss: 0.16908258199691772
Epoch 4639, Loss: 0.29705360531806946, Final Batch Loss: 0.1680937111377716
Epoch 4640, Loss: 0.237098827958107, Final Batch Loss: 0.1367906928062439
Epoch 4641, 

Epoch 4752, Loss: 0.23391831666231155, Final Batch Loss: 0.10725107043981552
Epoch 4753, Loss: 0.2413797527551651, Final Batch Loss: 0.06834663450717926
Epoch 4754, Loss: 0.24910999834537506, Final Batch Loss: 0.12400482594966888
Epoch 4755, Loss: 0.24998673051595688, Final Batch Loss: 0.13530948758125305
Epoch 4756, Loss: 0.2742750197649002, Final Batch Loss: 0.1396322399377823
Epoch 4757, Loss: 0.2880241423845291, Final Batch Loss: 0.1346760094165802
Epoch 4758, Loss: 0.2760499566793442, Final Batch Loss: 0.1686081886291504
Epoch 4759, Loss: 0.2778678983449936, Final Batch Loss: 0.1319492608308792
Epoch 4760, Loss: 0.2600501477718353, Final Batch Loss: 0.10302591323852539
Epoch 4761, Loss: 0.28955867886543274, Final Batch Loss: 0.16494335234165192
Epoch 4762, Loss: 0.23326408118009567, Final Batch Loss: 0.12123018503189087
Epoch 4763, Loss: 0.23719732463359833, Final Batch Loss: 0.11204126477241516
Epoch 4764, Loss: 0.24299712479114532, Final Batch Loss: 0.13099108636379242
Epoch 476

Epoch 4875, Loss: 0.2379748523235321, Final Batch Loss: 0.08487054705619812
Epoch 4876, Loss: 0.21940472722053528, Final Batch Loss: 0.07686467468738556
Epoch 4877, Loss: 0.2784995883703232, Final Batch Loss: 0.1452629417181015
Epoch 4878, Loss: 0.23596389591693878, Final Batch Loss: 0.12260337918996811
Epoch 4879, Loss: 0.26924092322587967, Final Batch Loss: 0.14860346913337708
Epoch 4880, Loss: 0.28359922021627426, Final Batch Loss: 0.18739593029022217
Epoch 4881, Loss: 0.20189018547534943, Final Batch Loss: 0.09767189621925354
Epoch 4882, Loss: 0.2128836140036583, Final Batch Loss: 0.07322227209806442
Epoch 4883, Loss: 0.23301928490400314, Final Batch Loss: 0.10111527889966965
Epoch 4884, Loss: 0.21720115840435028, Final Batch Loss: 0.08708122372627258
Epoch 4885, Loss: 0.2452443689107895, Final Batch Loss: 0.08419455587863922
Epoch 4886, Loss: 0.1967778503894806, Final Batch Loss: 0.08174130320549011
Epoch 4887, Loss: 0.2409558668732643, Final Batch Loss: 0.12070439755916595
Epoch 

Epoch 4982, Loss: 0.19325220584869385, Final Batch Loss: 0.09454800188541412
Epoch 4983, Loss: 0.17383059859275818, Final Batch Loss: 0.0881182923913002
Epoch 4984, Loss: 0.2427009865641594, Final Batch Loss: 0.10072367638349533
Epoch 4985, Loss: 0.2325357049703598, Final Batch Loss: 0.13621850311756134
Epoch 4986, Loss: 0.2838699221611023, Final Batch Loss: 0.15411975979804993
Epoch 4987, Loss: 0.2473076656460762, Final Batch Loss: 0.12275644391775131
Epoch 4988, Loss: 0.2797350138425827, Final Batch Loss: 0.1437150090932846
Epoch 4989, Loss: 0.20456843823194504, Final Batch Loss: 0.10762491077184677
Epoch 4990, Loss: 0.21720967441797256, Final Batch Loss: 0.07143130153417587
Epoch 4991, Loss: 0.26669611781835556, Final Batch Loss: 0.11088188737630844
Epoch 4992, Loss: 0.25098979473114014, Final Batch Loss: 0.10707665979862213
Epoch 4993, Loss: 0.2279892861843109, Final Batch Loss: 0.14592181146144867
Epoch 4994, Loss: 0.26798000931739807, Final Batch Loss: 0.12589074671268463
Epoch 4

In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[14  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  1]
 [ 0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0]
 [ 0  0  1  0  0 11  0  0  1]
 [ 0  0  0  0  0  0 12  0  0]
 [ 0  0  0  0  0  0  0  9  0]
 [ 0  0  1  0  0  0  0  0 14]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        14
           1    1.00000   1.00000   1.00000         9
           2    0.84615   0.91667   0.88000        12
           3    1.00000   1.00000   1.00000         7
           4    1.00000   1.00000   1.00000         6
           5    1.00000   0.84615   0.91667        13
           6    1.00000   1.00000   1.00000        12
           7    1.00000   1.00000   1.00000         9
           8    0.87500   0.93333   0.90323        15

    accuracy                        0.95876        97
   macro avg    0.96902   0.96624   0.96665        97
weighted avg    0.96164   0.95876   0.95902        97



In [24]:
n_samples = 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Group 3_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9))

fake_features = torch.Tensor(fake_features)

In [25]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[20  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  3  0  0  5]
 [ 0  0  0 19  1  0  0  0  0]
 [ 0  0  0  0 17  0  0  3  0]
 [ 0  0  4  0  0 15  0  0  1]
 [ 0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0 20  0]
 [ 0  0  2  0  0  3  0  0 15]]
              precision    recall  f1-score   support

         0.0    1.00000   1.00000   1.00000        20
         1.0    1.00000   1.00000   1.00000        20
         2.0    0.66667   0.60000   0.63158        20
         3.0    1.00000   0.95000   0.97436        20
         4.0    0.94444   0.85000   0.89474        20
         5.0    0.71429   0.75000   0.73171        20
         6.0    1.00000   1.00000   1.00000        20
         7.0    0.86957   1.00000   0.93023        20
         8.0    0.71429   0.75000   0.73171        20

    accuracy                        0.87778       180
   macro avg    0.87881   0.87778   0.87715       180
weighted avg    0.87881   0.87778   0.87715       180

